In [2]:
%load_ext autoreload
%autoreload 2

from dataloader import WICO
import tensorflow as tf
from model import Net
from spektral.data import DisjointLoader
from tensorflow import keras
from tqdm import tqdm
import numpy as np

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy

In [3]:
data = WICO("./dataset/WICO/", root_edges=False, time_delay_edges=False)

In [ ]:
def random_split(data, train_pct=0.75, train_epochs=5, seed=None):
    """ returns two DataLoaders, resp. for train and test set 
        according to the input split percentage """
    if seed is not None:
        np.random.default_rng(seed)

    np.random.shuffle(data)
    n = len(data)
    train_size, val_size = int(n*train_pct), int(val_pct*n)

    train_set = data[:train_size]
    test_set = data[train_size:]

    train_loader = DisjointLoader(train_set, batch_size=1, epochs=train_epochs)
    test_loader = DisjointLoader(test_set, batch_size=1, epochs=1)

    return train_loader, test_loader

In [4]:
np.random.default_rng(0)
np.random.shuffle(data)

train_pct = 0.2
val_pct = 0.1
num_epochs = 5

n = len(data)
train_size, val_size = int(n*train_pct), int(val_pct*n)

train_set = data[:train_size]
val_set = data[train_size:(train_size + val_size)]
test_set = data[(train_size + val_size):]

train_loader = DisjointLoader(train_set, batch_size=1, epochs=num_epochs)
val_loader = DisjointLoader(val_set, batch_size=1, epochs=1)
test_loader = DisjointLoader(test_set, batch_size=1, epochs=1)

In [1]:
model = Net(num_classes=3)
loss_fn = CategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=0.001)

def train_step(inputs, target):

    with tf.GradientTape() as tape:
        predictions, _, _ = model(inputs[:2])  # drop edge features from inputs
        loss = loss_fn(target, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc

NameError: name 'Net' is not defined

In [100]:
epoch = step = 0
results = []

for batch in tqdm(train_loader):
    step += 1
    loss, acc = train_step(*batch)
    results.append((loss, acc))

    if step == train_loader.steps_per_epoch:
        step = 0
        epoch += 1
        print("Ep. {} - Loss: {}. Acc: {}".format(epoch, *np.mean(results, 0)))
        results = []

698it [04:07,  2.48it/s]

Ep. 1 - Loss: 0.8249131441116333. Acc: 0.6962750554084778


1396it [08:12,  2.60it/s]

Ep. 2 - Loss: 0.8156884908676147. Acc: 0.7020057439804077


2094it [12:26,  2.59it/s]

Ep. 3 - Loss: 0.8137251734733582. Acc: 0.7020057439804077


2792it [16:43,  2.96it/s]

Ep. 4 - Loss: 0.8159133195877075. Acc: 0.7020057439804077


3490it [20:38,  2.82it/s]

Ep. 5 - Loss: 0.8146923780441284. Acc: 0.7020057439804077


In [103]:
results = []

for batch in test_loader:
    inputs, target = batch
    predictions, _, _ = model(inputs[:2], training=False)
    results.append(
        (
            loss_fn(target, predictions),
            tf.reduce_mean(categorical_accuracy(target, predictions)),
        )
    )
print("Done. Test loss: {}. Test acc: {}".format(*np.mean(results, 0)))

Done. Test loss: 0.7919934391975403. Test acc: 0.7187883853912354


In [64]:
train_loader.steps_per_epoch

15